In [16]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

% matplotlib inline

def sigmoid(z):
    s = 1.0/(1+np.exp(-z))
    return s


def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))

    return w, b


def propagate(w, b, X, Y):
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X)+b) 
    print(Y)
    print(A)
    print(Y.shape, A.shape, np.log(A).shape)
    print(np.log(A))
    print(Y*np.log(A))
    print((1-Y)*np.log(1-A))
    print(Y*np.log(A)+(1-Y)*np.log(1-A))
    cost = -(1.0/m)*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A)) 
    dw = (1.0/m)*np.dot(X,(A-Y).T)
    db = (1.0/m)*np.sum(A-Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())

    grads = {"dw": dw, "db": db}

    return grads, cost


def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):

    costs = []

    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)

        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate*dw
        b = b - learning_rate*db

        if i % 100 == 0:
            costs.append(cost)

        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" %(i, cost))

    params = {"w": w,
              "b": b}

    grads = {"dw": dw,
             "db": db}
        
        
    return params, grads, costs
        
    
def predict(w, b, X):

    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)

    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(m):
        if A[0,i] > 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
            
    assert(Y_prediction.shape == (1, m))

    return Y_prediction


def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    w, b = initialize_with_zeros(X_train.shape[0])
    
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)

    w = parameters["w"]
    b = parameters["b"]

    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))


    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}

    return d    
    
    
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

train_set_x_flatten = train_set_x_orig.reshape(m_train, -1).T
test_set_x_flatten = test_set_x_orig.reshape(m_test, -1).T

train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

    
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 50, learning_rate = 0.005, print_cost = True)


# learning_rates = [0.01, 0.001, 0.0001]
# models = {}
# for i in learning_rates:
#     print ("learning rate is: " + str(i))
#     models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
#     print ('\n' + "-------------------------------------------------------" + '\n')

# for i in learning_rates:
#     plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

# plt.ylabel('cost')
# plt.xlabel('iterations')

# legend = plt.legend(loc='upper center', shadow=True)
# frame = legend.get_frame()
# frame.set_facecolor('0.90')
# plt.show()


[[0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0
  0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1
  1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 0 0 1 0
  1 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1
  0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0]]
[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 

[[-1.62549128 -2.52696051 -2.64321932 -2.17280082 -1.87986473 -1.58174056
  -1.84133285 -1.78908669 -2.29432041 -2.08874575 -2.77447258 -2.45161669
  -1.90348507 -3.09639166 -2.84499407 -3.14873748 -3.1672127  -3.87449436
  -2.6763592  -1.30637406 -2.72215288 -1.96917664 -3.59941821 -3.15820295
  -2.6817311  -1.22719911 -2.50738147 -2.41997    -2.0719321  -2.41095566
  -1.64561668 -2.45660457 -2.21795895 -2.51800261 -2.28183088 -1.64787207
  -2.33259082 -3.36009854 -0.92565213 -2.003799   -2.49789279 -1.51075727
  -2.41443037 -2.89141316 -2.59495284 -1.63584803 -2.42435551 -1.80127286
  -2.47295168 -2.14212536 -3.15799578 -2.46582113 -2.70685826 -3.08438404
  -1.89249836 -3.3361714  -1.87201747 -3.42914884 -1.41117715 -1.56910982
  -1.55409153 -2.23934273 -2.99676725 -2.2088498  -3.4283597  -1.77632316
  -2.61522648 -2.35416524 -2.59165685 -2.91724116 -2.50032173 -1.70208375
  -1.60728125 -1.71475769 -2.28208811 -2.6597479  -2.45282715 -2.14890419
  -2.21225609 -1.35866007 -2.81801937 

  0.03990439 0.30583868 0.08428114 0.30652351 0.20015411]]
(1, 209) (1, 209) (1, 209)
[[-1.56815386 -2.56078163 -2.46345076 -2.61720087 -2.01332138 -1.59098
  -1.98457797 -1.65424347 -2.3418271  -2.14605861 -2.62643379 -2.41640805
  -1.84985245 -3.0058086  -2.89868705 -3.11417357 -3.27486172 -4.05412903
  -2.67031793 -1.15852394 -2.8436738  -2.13274314 -3.52963939 -3.04939778
  -2.53537827 -1.19174321 -2.6509044  -2.37787925 -2.17851978 -2.5366192
  -1.7648247  -2.29827603 -2.03112193 -2.4072631  -2.49898654 -1.5504228
  -2.4528384  -3.3027111  -0.89204318 -1.8872018  -2.69078339 -1.65888044
  -2.2836833  -2.94010093 -2.75610912 -1.62866213 -2.39829419 -1.70032423
  -2.88717644 -2.10143373 -3.02519547 -2.38892449 -2.72532169 -2.88318676
  -1.84439764 -3.44533804 -1.67958235 -3.21791163 -1.47689778 -1.40944784
  -1.38679131 -2.17811539 -3.13180038 -2.27085062 -3.32357405 -1.78043586
  -2.47508429 -2.38092386 -2.52282958 -2.95206056 -2.43480461 -1.49691251
  -1.71861056 -1.56401356 -2.20

  -0.32546175 -0.83790236 -0.4681619  -0.90148476 -0.61345719]]
[[-0.         -0.         -0.29663112 -0.         -0.         -0.
  -0.         -0.40552057 -0.         -0.         -0.         -0.50838289
  -0.         -0.42184772 -0.6131716  -0.         -0.         -0.
  -0.         -0.36598353 -0.         -0.         -0.         -0.
  -0.3443395  -0.57455161 -0.         -0.48844922 -0.         -0.80403588
  -0.         -0.         -0.         -0.         -0.         -0.
  -0.         -0.         -0.57725756 -0.         -0.         -1.0023367
  -0.36426161 -0.         -0.         -0.         -0.         -0.40640914
  -0.         -0.         -0.32782566 -0.         -0.         -0.
  -0.5148807  -0.         -0.3065789  -0.24400137 -0.         -0.35275344
  -0.31913339 -0.47950661 -0.         -0.         -0.         -0.
  -0.         -0.         -0.42573109 -0.         -0.         -0.29545032
  -0.         -0.         -0.         -0.         -0.         -0.
  -0.         -0.         -0.  

In [15]:
a = np.array([[1,2,3]])
b = np.array([[1,2,3]])
print(a.shape, b.shape)
print(a*b)

(1, 3) (1, 3)
[[1 4 9]]
